In [5]:
# Import necessary libraries
import requests
 
# Define the URL of the deployed endpoint
url = "http://127.0.0.1:8000"
 
def test_ner_service(input_text):
    """
    Function to send a POST request to the NER service and print the results.
    """
    response = requests.post(url, data={"text": input_text})
    if response.status_code == 200:
        print("Response received successfully.")
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        results = soup.find_all('li')
        for result in results:
            print(result.text)
    else:
        print(f"Failed to get response. Status code: {response.status_code}")
 
# Test the service with some example text
test_ner_service("Polyethylene terephthalate")

Response received successfully.
poly - Longform
##eth - Longform
##yle - Longform
##ne - Longform
ter - Longform
##ep - Longform
##ht - Longform
##hala - Longform
##te - Longform


In [6]:
#response time
import time
 
def measure_response_time(text):
    start_time = time.time()
    response = requests.post(url, data={"text": text})
    end_time = time.time()
    return end_time - start_time, response.status_code
 
# Generate texts of varying lengths
short_text = "text"
medium_text = "this is a very quite extremely not long piece of text"
long_text = "this is a very quite extremely not long piece of text" * 10
very_long_text = "this is a very quite extremely not long piece of text" * 100
very_longer_text = "test " * 510

# Measure response times
short_time, short_status = measure_response_time(short_text)
medium_time, medium_status = measure_response_time(medium_text)
long_time, long_status = measure_response_time(long_text)
very_long_time, very_long_status = measure_response_time(very_long_text)
very_longer_time, very_longer_status = measure_response_time(very_longer_text)
 
# Print results
print(f"Short text response time: {short_time:.2f} seconds, Status code: {short_status}")
print(f"Medium text response time: {medium_time:.2f} seconds, Status code: {medium_status}")
print(f"Long text response time: {long_time:.2f} seconds, Status code: {long_status}")
print(f"Very long text response time: {very_long_time:.2f} seconds, Status code: {very_long_status}")
print(f"Very longer text response time: {very_longer_time:.2f} seconds, Status code: {very_longer_status}")

Short text response time: 0.03 seconds, Status code: 200
Medium text response time: 0.02 seconds, Status code: 200
Long text response time: 0.08 seconds, Status code: 200
Very long text response time: 0.22 seconds, Status code: 500
Very longer text response time: 0.38 seconds, Status code: 200


In [4]:
#concurrency-
import concurrent.futures
 
def concurrent_test(text, num_requests):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_request = {executor.submit(measure_response_time, text): i for i in range(num_requests)}
        results = []
        for future in concurrent.futures.as_completed(future_to_request):
            results.append(future.result())
        return results
 
# Number of concurrent requests
num_requests = 50
 
# Perform concurrency test
concurrent_results = concurrent_test(very_longer_text, num_requests)
 
# Calculate average response time and success rate
average_time = sum(result[0] for result in concurrent_results) / num_requests
success_rate = sum(1 for result in concurrent_results if result[1] == 200) / num_requests
 
# Print results
print(f"Average response time for {num_requests} concurrent requests: {average_time:.2f} seconds")
print(f"Success rate for {num_requests} concurrent requests: {success_rate * 100:.2f}%")

Average response time for 1000 concurrent requests: 0.57 seconds
Success rate for 1000 concurrent requests: 100.00%
